In [33]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Connection details
username = 'lol_user'
password = 'your_password'
host = '35.203.46.85'
port = '3306'
database = 'lol_retention'

# Create engine
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# Test the connection
try:
    connection = engine.connect()
    print("Connected successfully!")
except Exception as e:
    print(f"Error: {e}")

# Define your queries with `rank` escaped
summoner_details_query = "SELECT * FROM summoner_details"
match_details_query = "SELECT * FROM match_details"
entries_query = "SELECT summoner_id, `rank` FROM entries"  # Using backticks around `rank`

# Load the data into pandas dataframes
summoner_details_df = pd.read_sql_query(summoner_details_query, engine)
match_details_df = pd.read_sql_query(match_details_query, engine)
entries_df = pd.read_sql_query(entries_query, engine)

# Filter out rows in match_details that correspond to players in summoner_details
target_summoners = summoner_details_df['summoner_id'].unique()
filtered_match_details = match_details_df[match_details_df['summoner_id'].isin(target_summoners)].copy()

# Convert game_creation_converted to datetime and calculate the difference in days
filtered_match_details['game_creation_converted'] = pd.to_datetime(filtered_match_details['game_creation_converted'])
today = datetime.now()
filtered_match_details['days_diff'] = (today - filtered_match_details['game_creation_converted']).dt.days

# Define the groups based on days_diff
def assign_time_group(days):
    if days <= 7:
        return 'Active Players (1-20 days)'
    elif 8 <= days <= 14:
        return 'Slightly Inactive (21-40 days)'
    elif 15 <= days <= 28:
        return 'Moderately Inactive (41-60 days)'
    elif 29 <= days <= 40:
        return 'Highly Inactive (61-80 days)'
    elif 41 <= days <= 60:
        return 'At Risk of Churn (81-100 days)'
    else:
        return 'Beyond 100 days - Lost'

# Apply the function to create the time groups
filtered_match_details['time_group'] = filtered_match_details['days_diff'].apply(assign_time_group)

# Joining with the entries DataFrame to get the rank information
combined_data = pd.merge(filtered_match_details, entries_df, on='summoner_id', how='left')

# Group by the new time_group and rank, and count the number of results in each group
grouped_results = combined_data.groupby(['time_group', 'rank']).size().reset_index(name='count')
print(grouped_results)


Connected successfully!
                          time_group rank  count
0         Active Players (1-20 days)    I    585
1         Active Players (1-20 days)   II    651
2         Active Players (1-20 days)  III    629
3         Active Players (1-20 days)   IV    623
4     At Risk of Churn (81-100 days)    I     65
5     At Risk of Churn (81-100 days)   II     91
6     At Risk of Churn (81-100 days)  III     90
7     At Risk of Churn (81-100 days)   IV     76
8             Beyond 100 days - Lost    I    127
9             Beyond 100 days - Lost   II     77
10            Beyond 100 days - Lost  III     61
11            Beyond 100 days - Lost   IV     87
12      Highly Inactive (61-80 days)    I     73
13      Highly Inactive (61-80 days)   II     56
14      Highly Inactive (61-80 days)  III     63
15      Highly Inactive (61-80 days)   IV     66
16  Moderately Inactive (41-60 days)    I    123
17  Moderately Inactive (41-60 days)   II    100
18  Moderately Inactive (41-60 days)  III    